# Regression Week 2: Multiple Regression (Interpretation)

The goal of this first notebook is to explore multiple regression and feature engineering with existing graphlab functions.

In this notebook you will use data on house sales in King County to predict prices using multiple regression. You will:
* Use SFrames to do some feature engineering
* Use built-in graphlab functions to compute the regression weights (coefficients/parameters)
* Given the regression weights, predictors and outcome write a function to compute the Residual Sum of Squares
* Look at coefficients and interpret their meanings
* Evaluate multiple models via RSS

# Fire up graphlab create

In [5]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.linear_model import LinearRegression

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
train = pd.read_csv('kc_house_train_data.csv')
test = pd.read_csv('kc_house_test_data.csv')

# Split data into training and testing.
We use seed=0 so that everyone running this notebook gets the same results.  In practice, you may set a random seed (or let GraphLab Create pick a random seed for you).  

# Learning a multiple regression model

Recall we can use the following code to learn a multiple regression model predicting 'price' based on the following features:
example_features = ['sqft_living', 'bedrooms', 'bathrooms'] on training data with the following code:

(Aside: We set validation_set = None to ensure that the results are always the same)

In [8]:
example_features = ['sqft_living', 'bedrooms', 'bathrooms']
example_model = LinearRegression().fit(train.loc[:,example_features], train['price'])

Now that we have fitted the model we can extract the regression weights (coefficients) as an SFrame as follows:

In [9]:
example_model.coef_

array([   315.40669062, -65081.88711588,   6942.16598637])

# Making Predictions

In the gradient descent notebook we use numpy to do our regression. In this book we will use existing graphlab create functions to analyze multiple regressions. 

Recall that once a model is built we can use the .predict() function to find the predicted values for data we pass. For example using the example model above:

In [11]:
example_model.predict(train.loc[:,example_features])[0] # should be 271789.505878

271789.26537996985

# Compute RSS

Now that we can make predictions given the model, let's write a function to compute the RSS of the model. Complete the function below to calculate RSS given the model, data, and the outcome.

In [20]:
def get_residual_sum_of_squares(model, data, outcome):
    # First get the predictions
    y_pred = model.predict(data)
    # Then compute the residuals/errors
    rss = (outcome - y_pred)**2
    # Then square and add them up
    RSS = np.sum(rss)
    return(RSS) 

Test your function by computing the RSS on TEST data for the example model:

In [21]:
rss_example_train = get_residual_sum_of_squares(example_model, test.loc[:,example_features], test['price'])
print(rss_example_train) # should be 2.7376153833e+14

273761940583133.72


# Create some new features

Although we often think of multiple regression as including multiple different features (e.g. # of bedrooms, squarefeet, and # of bathrooms) but we can also consider transformations of existing features e.g. the log of the squarefeet or even "interaction" features such as the product of bedrooms and bathrooms.

You will use the logarithm function to create a new feature. so first you should import it from the math library.

In [22]:
from math import log

Next create the following 4 new features as column in both TEST and TRAIN data:
* bedrooms_squared = bedrooms\*bedrooms
* bed_bath_rooms = bedrooms\*bathrooms
* log_sqft_living = log(sqft_living)
* lat_plus_long = lat + long 
As an example here's the first one:

In [24]:
train['bedrooms_squared'] = train['bedrooms'].apply(lambda x: x**2)
test['bedrooms_squared'] = test['bedrooms'].apply(lambda x: x**2)

In [28]:
# create the remaining 3 features in both TEST and TRAIN data
train['bed_bath_rooms'] = train['bedrooms']*train['bathrooms']
test['bed_bath_rooms'] = test['bedrooms']*test['bathrooms']
train['log_sqft_living'] = train['sqft_living'].apply(lambda x: log(x))
test['log_sqft_living'] = test['sqft_living'].apply(lambda x: log(x))
train['lat_plus_long'] = train['lat']+train['long']
test['lat_plus_long'] = test['lat']+test['long']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this feature will mostly affect houses with many bedrooms.
* bedrooms times bathrooms gives what's called an "interaction" feature. It is large when *both* of them are large.
* Taking the log of squarefeet has the effect of bringing large values closer together and spreading out small values.
* Adding latitude to longitude is totally non-sensical but we will do it anyway (you'll see why)

**Quiz Question: What is the mean (arithmetic average) value of your 4 new features on TEST data? (round to 2 digits)**

In [31]:
print(test.bedrooms_squared.mean())
print(test.bed_bath_rooms.mean())
print(test.log_sqft_living.mean())
print(test.lat_plus_long.mean())

12.4466777015843
7.5039016315913925
7.550274679645921
-74.65333355403185


# Learning Multiple Models

Now we will learn the weights for three (nested) models for predicting house prices. The first model will have the fewest features the second model will add one more feature and the third will add a few more:
* Model 1: squarefeet, # bedrooms, # bathrooms, latitude & longitude
* Model 2: add bedrooms\*bathrooms
* Model 3: Add log squarefeet, bedrooms squared, and the (nonsensical) latitude + longitude

In [32]:
model_1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model_2_features = model_1_features + ['bed_bath_rooms']
model_3_features = model_2_features + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']

Now that you have the features, learn the weights for the three different models for predicting target = 'price' using graphlab.linear_regression.create() and look at the value of the weights/coefficients:

In [46]:
# Learn the three models: (don't forget to set validation_set = None)
model1 = LinearRegression().fit(train.loc[:,model_1_features], train['price'])
model2 = LinearRegression().fit(train.loc[:,model_2_features], train['price'])
mode1_last = LinearRegression().fit(train.loc[:,model_3_features], train['price'])

In [47]:
# Examine/extract each model's coefficients:
print(model1.coef_)
print(model2.coef_)
print(mode1_last.coef_)

[ 3.12258646e+02 -5.95865332e+04  1.57067421e+04  6.58619264e+05
 -3.09374351e+05]
[ 3.06610053e+02 -1.13446368e+05 -7.14613083e+04  6.54844630e+05
 -2.94298969e+05  2.55796520e+04]
[ 5.29422820e+02  3.45142296e+04  6.70607813e+04  5.34085611e+05
 -4.06750711e+05 -8.57050439e+03 -6.78858667e+03 -5.61831484e+05
  1.27334900e+05]


In [49]:
print(get_residual_sum_of_squares(model1, test.loc[:,model_1_features], test['price']))
print(get_residual_sum_of_squares(model2, test.loc[:,model_2_features], test['price']))
print(get_residual_sum_of_squares(mode1_last, test.loc[:,model_3_features], test['price']))

225500469795490.34
223377462976467.5
259236319207179.97


**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 1?**

**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 2?**

Think about what this means.

# Comparing multiple models

Now that you've learned three models and extracted the model weights we want to evaluate which model is best.

First use your functions from earlier to compute the RSS on TRAINING Data for each of the three models.

In [ ]:
# Compute the RSS on TRAINING data for each of the three models and record the values:
print(get_residual_sum_of_squares(model1, train.loc[:,model_1_features], train['price']))
print(get_residual_sum_of_squares(model2, train.loc[:,model_2_features], train['price']))
print(get_residual_sum_of_squares(mode1_last, train.loc[:,model_3_features], train['price']))

**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TRAINING Data?** Is this what you expected?

Now compute the RSS on on TEST data for each of the three models.

In [ ]:
# Compute the RSS on TESTING data for each of the three models and record the values:
print(get_residual_sum_of_squares(model1, test.loc[:,model_1_features], test['price']))
print(get_residual_sum_of_squares(model2, test.loc[:,model_2_features], test['price']))
print(get_residual_sum_of_squares(mode1_last, test.loc[:,model_3_features], test['price']))

**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TESTING Data?** Is this what you expected? Think about the features that were added to each model from the previous.